In [33]:

%pip install tabula-py
from tabula import read_pdf as reader
import tabula
import numpy as np
import pandas as pd
import re

Note: you may need to restart the kernel to use updated packages.


In [32]:
pages=np.arange(1,44)
# reason we converted to the list is due to the fact that np array has type any 
pages=pages.tolist()
df=pd.DataFrame()

for page in pages:
    df1=tabula.read_pdf("CBT.pdf",pages=page)
    df1=df1[0]
    # type conversion to the int for comparsion
    df1=df1.dropna()
    # we used concat instead of append due error throw
    df=pd.concat([df,df1],ignore_index=True)


def check(value):
    if type(value)!= int or type(value)!= float: print(value)

def customtoint(value):
    pattern= r"\d \*"
    replacement=r'\d'
    re.sub(pattern,replacement,value)
    return int(value)
df["CBT Score"]=df["CBT Score"].apply(toint)
df["NAN"]=df["CBT Score"].apply(check)


ValueError: invalid literal for int() with base 10: '601 *'

In [24]:

# no need for if condition can be done directly
df=df[df["Applied Program"] =="BE in Computer Engineering"]
df.head()
df=df.sort_values(by="CBT Score",ascending=0)
df["Rank"]=df["CBT Score"].rank(ascending=False)
df.reset_index(drop=True)

TypeError: '<' not supported between instances of 'float' and 'str'